In [31]:
import dataiku
import pandas as pd
import mlflow

from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, StratifiedKFold
from dataikuapi.dss.ml import DSSPredictionMLTaskSettings
from sklearn.model_selection import GridSearchCV

In [32]:
# Replace these constants by your own values
XP_TRACKING_FOLDER_ID = "E4Kae8lm"
MLFLOW_EXPERIMENT_NAME = "uci-banking-demo"
MLFLOW_CODE_ENV_NAME = "mlflow-env"
SAVED_MODEL_NAME = "uci-bank-clf"
EVALUATION_DATASET = "uci_bank_evaluate"

In [33]:
# Some utils
def now_str() -> str:
    return datetime.now().strftime("%Y%m%d%H%M%S")

# Experiment tracking (scikit-learn)

This notebook contains a simple example to showcase the new Experiment Tracking capabilities of Dataiku. It explains how to perform several runs with different parameters, select the best run and promote it as a Saved Model version in a Dataiku Flow. It leverages:
* the [scikit-learn]() package
* the [UCI Bank Marketing dataset]()

## 1. Loading the training data

Our training data lives in the `uci_bank_train` Dataset, let's load it in a pandas DataFrame and see what it looks like:

In [34]:
client = dataiku.api_client()
project = client.get_default_project()
training_dataset = dataiku.Dataset("UCI_BANK_TRAIN")
df = training_dataset.get_dataframe()
df.head()

/data/dataiku/dss_data/code-envs/python/Python38/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
4,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


We are working on a * binary classification* problem here, which is to predict whether or not a given person who was part of a marketing campaign ended up purchasing one of the bank's products. This outcome is reflected by the `y` column which can either take the "no" or "yes" values.

In [35]:
target_name = "y"
target = df[target_name]
data = df.drop(columns=[target_name])

/data/dataiku/dss_data/code-envs/python/Python38/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


## 2. Preparing the experiment

To prepare the grounds for our experiments, we need to create a few handles and define which MLFlow experiment we'll collect our runs into:

In [36]:
# Create a mlflow_extension object to easily collect information for the promotion step
mlflow_extension = project.get_mlflow_extension()

# Get a handle on the Managed Folder that will contain the experiment run + artifact data
# TODO Get-or-create managed folder
folder = project.get_managed_folder(odb_id=XP_TRACKING_FOLDER_ID)

# Create a handle for the mlflow client
mlflow_handle = project.setup_mlflow(managed_folder=folder)

# Set the experiment
mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)
mlflow_experiment = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

## 3. Experimenting

The goal of experiment tracking is to *instrument the iterative process of ML model training* by collecting all parameters and results of each trial. To be more specific, within an **experiment**, you perform multiple **runs**, each run being different from the others because of the **parameters** you use for it. You also need to specific which **metrics** to track, they will reflect the performance of the model for a given set of parameters.

In this notebook example, if you want to produce experiment runs:
* edit the parameters in the 3.1 cell and run it
* run the 3.2 cell to effectively... perform the run 🙂

### 3.1 Defining the parameters of our run

In [37]:
# Create run name
run_name = f"run-{now_str()}"
run_params = {}
run_metrics = {}

# Define run parameters
# -- Which categorical columns to retain ?
categorical_cols = ["job",
                    "marital",
                    "education",
                    "default",
                    "housing",
                    "loan",
                    "contact",
                    "month",
                    "poutcome"]
run_params["categorical_cols"] = categorical_cols

# --Which algorithm to use? Which hyperparameters for this algo to try?
# --- Example: Random Forest
hparams = {"n_estimators": 100,
           "criterion": "gini",
           "max_depth": 6,
           "min_samples_split": 5,
           "random_state": 42}

clf = RandomForestClassifier()
"""
grid_rf_model = GridSearchCV(rf, parameters, cv=3)
grid_rf_model.fit(X_train, y_train)

best_rf = grid_rf_model.best_estimator_
for p in parameters:
  print("Best '{}': {}".format(p, best_rf.get_params()[p]))



rf = RandomForestClassifier()
clf = GridSearchCV(rf, hparams, cv=3)
"""

# ---Example: Gradient Boosting
#hparams = {"n_estimators": 300,
#           "loss": "exponential",
#           "learning_rate": 0.1,
#           "max_depth": 3,
#           "random_state": 42}
#clf = GradientBoostingClassifier(**hparams)
model_algo = type(clf).__name__
run_params["model_algo"] = model_algo
for hp in hparams.keys():
    run_params[hp] = hparams[hp]

# --Which cross-validation settings to use?
n_cv_folds = 3
cv = StratifiedKFold(n_splits=n_cv_folds)
run_params["n_cv_folds"] = n_cv_folds
metrics = ["f1_macro", "roc_auc"]

# --Let's print all of that to get a recap:
print(f"Parameters to log:\n {run_params}")
print(100*'-')
print(f"Metrics to log:\n {metrics}")

Parameters to log:
 {'categorical_cols': ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'], 'model_algo': 'RandomForestClassifier', 'n_estimators': 100, 'criterion': 'gini', 'max_depth': 6, 'min_samples_split': 5, 'random_state': 42, 'n_cv_folds': 3}
----------------------------------------------------------------------------------------------------
Metrics to log:
 ['f1_macro', 'roc_auc']


### 3.2 Performing the run and logging parameters, metrics and the model

In [38]:
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    print(f"Starting run {run_name} (id: {run_id})...")
    # --Preprocessing
    categorical_preprocessor = OrdinalEncoder()
    preprocessor = ColumnTransformer([('categorical', categorical_preprocessor, categorical_cols)],
                                     remainder="passthrough")

    # --Pipeline definition (preprocessing + model)
    pipeline = make_pipeline(preprocessor, clf)

    # --Cross-validation
    print(f"Running cross-validation...")
    scores = cross_validate(pipeline, data, target, cv=cv, scoring=metrics)
    for m in [f"test_{mname}" for mname in metrics]:
        run_metrics[f"mean_{m}"] = scores[m].mean()
        run_metrics[f"std_{m}"] = scores[m].std()

    # --Pipeline fit
    pipeline.fit(X=data, y=target)
    # --Log the order of the class label
    run_params["class_labels"] = pipeline.classes_.tolist()

    # --Log parameters, metrics and model
    mlflow.log_params(params=run_params)
    mlflow.log_metrics(metrics=run_metrics)
    artifact_path = f"{model_algo}-{run_id}"
    mlflow.sklearn.log_model(sk_model=pipeline, artifact_path=artifact_path)

    # --Set useful information to faciliate run promotion
    mlflow_extension.set_run_inference_info(run_id=run_id,
                                            prediction_type="BINARY_CLASSIFICATION",
                                            classes=run_params["class_labels"],
                                            code_env_name=MLFLOW_EXPERIMENT_NAME,
                                            target="y")
    print(f"DONE! Your artifacts are available at {run.info.artifact_uri}")

Starting run run-20230224005222 (id: run_20230224005222)...
Running cross-validation...


/data/dataiku/dss_data/code-envs/python/Python38/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):
/data/dataiku/dss_data/code-envs/python/Python38/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.

DONE! Your artifacts are available at dss-managed-folder://E4Kae8lm/uci_banking_demo/run_20230224005222/artifacts


## 4. Promote a run to a Saved Model version

Now that you have tried several parameters and performed multiple runs, you may want to choose the "best" one and actually surface it on your Dataiku Flow. This is done by *promoting* an experiment run into a Saved Model version. Let's start by figuring out which experiment got the best results:

In [39]:
# List all runs of the experiment with performance metrics
run_cv_metrics = {}
for run_info in mlflow.list_run_infos(experiment_id=mlflow_experiment.experiment_id):
    run = mlflow.get_run(run_info.run_id)
    run_cv_metrics[run_info.run_id] = run.data.metrics

In [40]:
# Look for the best run according to a given metric:
metric_for_promotion = "mean_test_roc_auc"
# --Only keep the runs where the metric for promotion was logged:
simplified_run_cv_metrics = {}
for rid in run_cv_metrics.keys():
    if metric_for_promotion in run_cv_metrics[rid].keys():
        simplified_run_cv_metrics[rid] = run_cv_metrics[rid][metric_for_promotion]
best_run_id = sorted(simplified_run_cv_metrics)[0]
print(f"The best run is {best_run_id} with a {metric_for_promotion} of {simplified_run_cv_metrics[best_run_id]}")

The best run is run_20230224004210 with a mean_test_roc_auc of 0.5403114819487171


In [41]:
simplified_run_cv_metrics

{'run_20230224005222': 0.5278375524347928,
 'run_20230224004210': 0.5403114819487171}

In [42]:
# Retrieve the path of its model directory
best_run_info = mlflow.get_run(best_run_id).info
model_path = f"{mlflow_experiment.experiment_id}/{best_run_id}/artifacts/{model_algo}-{best_run_id}"
print(f"Its model is located at: {model_path}")

Its model is located at: uci_banking_demo/run_20230224004210/artifacts/RandomForestClassifier-run_20230224004210


In [43]:
best_run_info

<RunInfo: artifact_uri='dss-managed-folder://E4Kae8lm/uci_banking_demo/run_20230224004210/artifacts', end_time=1677199348423, experiment_id='uci_banking_demo', lifecycle_stage='active', run_id='run_20230224004210', run_uuid='run_20230224004210', start_time=1677199332644, status='FINISHED', user_id='dssuser_kirsten_da4e5f91'>

In [44]:
run.info.artifact_uri

'dss-managed-folder://E4Kae8lm/uci_banking_demo/run_20230224004210/artifacts'

Next, let's retrieve a handle for the Saved Model in which we are going to create our version:

In [45]:
# Get or create the Saved Model
sm_id = None
for sm in project.list_saved_models():
    if sm["name"] != SAVED_MODEL_NAME:
        continue
    else:
        sm_id = sm["id"]
        print(f"Found Saved Model {sm['name']} with id {sm['id']}")
        break
if sm_id:
    sm = project.get_saved_model(sm_id)
else:
    sm = project.create_mlflow_pyfunc_model(name=SAVED_MODEL_NAME,
                                            prediction_type=DSSPredictionMLTaskSettings.PredictionTypes.BINARY)
    sm_id = sm.id
    print(f"Saved Model not found, created new one with id {sm_id}")

Found Saved Model uci-bank-clf with id nvVYOuGW


We can now promote our experiment run and generate the corresponding Saved Model version:

In [48]:
# Import model by generating a Saved Model version
version_id = f"{best_run_id}_{now_str()}"
mlflow_version = sm.import_mlflow_version_from_managed_folder(version_id=version_id,
                                                              managed_folder=XP_TRACKING_FOLDER_ID,
                                                              path=model_path)
# Make this Saved Model version the active one
sm.set_active_version(mlflow_version.version_id)

Finally, to be able to visualize the performance graphs of our newly-created Saved Model version, we need to *evaluate* it against an evaluation Dataset. In our case, it's called `uci_bank_evaluate`:

In [49]:
# Evaluate the imported model
mlflow_version.set_core_metadata(target_column_name=target_name,
                                 class_labels=["no", "yes"], # TODO change with run params
                                 get_features_from_dataset=EVALUATION_DATASET)
mlflow_version.evaluate(EVALUATION_DATASET)

DataikuException: com.dataiku.dip.server.controllers.NotFoundException: dataset does not exist: EMERSONEVAL.uci_bank_evaluate

Congratulations! You now have a Saved Model version coming from a fully programmatic workflow!